In [21]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import random
import pickle
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import load_img
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder






In [13]:
Train_dir="images/train"
Test_dir="images/validation"

In [8]:
def createData(dir):
    image_paths=[]
    labels=[]
    for folder in os.listdir(dir):
        for file in os.listdir(os.path.join(dir,folder)):
            image_paths.append(os.path.join(dir,folder,file))
            labels.append(folder)
        print(folder," Completed")
    return image_paths,labels

In [9]:
train=pd.DataFrame()
train["path"],train["label"]=createData(Train_dir)

angry  Completed
disgust  Completed
fear  Completed
happy  Completed
neutral  Completed
sad  Completed
surprise  Completed


In [14]:
test=pd.DataFrame()
test["path"],test["label"]=createData(Test_dir)

angry  Completed
disgust  Completed
fear  Completed
happy  Completed
neutral  Completed
sad  Completed
surprise  Completed


In [17]:
def extractFeatures(df):
    features=[]
    for path in tqdm(df["path"]):
        img=load_img(path,grayscale=True)
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return np.array(features)

In [18]:
train_features=extractFeatures(train)

  0%|          | 0/28821 [00:00<?, ?it/s]

c:\Python310\lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [19]:
test_festures=extractFeatures(test)

  0%|          | 0/7066 [00:00<?, ?it/s]

c:\Python310\lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [20]:
x_train=train_features/255.0
x_test=test_festures/255.0

In [22]:
label_encoder=LabelEncoder()
label_encoder.fit(train["label"])

c:\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


LabelEncoder()

In [23]:
y_train=label_encoder.transform(train["label"])
y_test=label_encoder.transform(test["label"])
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

c:\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [30]:
model=Sequential()
# Convolutional layers(to get the features of the image)
model.add(Conv2D(128,(3,3),activation="relu",input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Flatten the output of convolutional layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7,activation="softmax"))



In [31]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [34]:
model.fit(x_train,y_train,batch_size=128,epochs=10,validation_data=(x_test,y_test))

Epoch 1/50
226/226 [==============================] - ETA: 0s - loss: 1.8178 - accuracy: 0.2469WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x00000230F09E5870> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
226/226 [==============================] - 242s 1s/step - loss: 1.

KeyboardInterrupt: 